In [ ]:
# ==========================================
# CELL 1: SETUP & DEPENDENCIES (CRASH PROOF)
# ==========================================
# Sabse pehle purana torchaudio hatayenge aur stable wala dalenge
# Taaki wo "TorchCodec" wala error dobara na aaye.
import os

print("⏳ Installing Dependencies... (Thoda time lagega)")
os.system("pip uninstall -y torchaudio")
os.system("pip install -q torchaudio==2.5.1 soundfile") 
os.system("pip install -q demucs fastapi uvicorn pyngrok python-multipart nest-asyncio")
print("✅ Installation Complete! Ready to Rock.")

In [ ]:

# ==========================================
# CELL 2: THE SERVER LOGIC (ULTRA QUALITY)
# ==========================================
import shutil
import subprocess
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import FileResponse
from pyngrok import ngrok
import uvicorn
import nest_asyncio

app = FastAPI()

def clean_environment():
    """Purana kachra saaf karne ke liye helper function"""
    paths = ["separated", "stems.zip", "input_song.mp3"]
    for path in paths:
        if os.path.exists(path):
            if os.path.isdir(path):
                shutil.rmtree(path)
            else:
                os.remove(path)

@app.post("/separate")
async def separate_audio(file: UploadFile = File(...)):
    try:
        # 1. Clean previous runs
        clean_environment()

        # 2. Save incoming file
        print(f"🎵 Receiving File: {file.filename}")
        with open("input_song.mp3", "wb") as buffer:
            shutil.copyfileobj(file.file, buffer)

        # 3. THE MAGIC COMMAND (High Quality Settings)
        # -n htdemucs_ft : Best Fine-Tuned Model
        # --shifts=4     : Accuracy ke liye multiple passes
        # --overlap=0.4  : Smooth cuts ke liye cross-fading
        print("🚀 Starting Demucs Separation on GPU...")
        command = [
            "demucs", 
            "-n", "htdemucs_ft", 
            "-d", "cuda", 
            "--shifts=4", 
            "--overlap=0.4", 
            "--jobs=2", 
            "input_song.mp3"
        ]
        
        # Subprocess use kar rahe hain taaki agar VRAM full ho toh crash handle ho jaye
        result = subprocess.run(command, capture_output=True, text=True)
        
        # Check for errors in Demucs output
        if result.returncode != 0:
            print("❌ Demucs Error Log:", result.stderr)
            raise Exception("Demucs processing failed")
            
        print("✅ Separation Complete!")

        # 4. Folder Detection (Smart Path Finding)
        # Kabhi kabhi folder 'htdemucs' hota hai, kabhi 'htdemucs_ft'
        base_path = "separated/htdemucs_ft"
        if not os.path.exists(base_path):
            base_path = "separated/htdemucs"
            
        if not os.path.exists(base_path):
            raise Exception("Output folder not found!")

        # Gane ka folder dhoondo (kyunki Demucs folder ka naam input file se leta hai)
        song_folders = os.listdir(base_path)
        if not song_folders:
            raise Exception("No separated songs found inside output folder")
            
        target_folder = os.path.join(base_path, song_folders[0])

        # 5. Zip & Send
        print("📦 Zipping files...")
        shutil.make_archive("stems", 'zip', target_folder)
        return FileResponse("stems.zip", media_type="application/zip", filename="stems.zip")

    except Exception as e:
        print(f"❌ Critical Error: {str(e)}")
        return {"error": str(e)}

@app.get("/")
def home():
    return {"status": "SonicSplit Ultra-Quality Server is Running 🚀"}

In [ ]:

# ==========================================
# CELL 3: LAUNCH SERVER
# ==========================================
# Apna NGROK Token yahan daal dena
NGROK_TOKEN = "37e2oaTWNuehV4hxwUue7xsJpYO_2NmfPuzohBbnYcTkKGRnK" 
ngrok.set_auth_token(NGROK_TOKEN)

# Port Tunneling
try:
    ngrok_tunnel = ngrok.connect(8000)
    print(f"\n💎 PUBLIC URL: {ngrok_tunnel.public_url} \n")
    print("👆 Copy this URL and paste it in your Flutter App")
    
    # Start Server
    config = uvicorn.Config(app, port=8000)
    server = uvicorn.Server(config)
    await server.serve()
except Exception as e:
    print(f"Server Error: {e}")